<a href="https://colab.research.google.com/github/edygordo/sns/blob/master/Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle

def store_data():
    db = direct_path
    dbfile = open('/content/drive/MyDrive/DPM/Direct_path_dict_full','ab')
    pickle.dump(db,dbfile)
    dbfile.close()
    
def load_data():
    dbfile = open('/content/drive/MyDrive/DPM/Direct_path_dict_full','rb')
    db = pickle.load(dbfile)
    dbfile.close()
    return db

In [3]:
def list_of_walks(lat, lon):
    file = open('/content/drive/MyDrive/DPM/stops.csv','r')
    my_list = []
    file.readline()
    while True:
        line = file.readline()
        if len(line)==0:
            break;
        else:
            tokens = line.split(',')
            f_lat = float(tokens[-2])
            f_lon = float(tokens[-1])
            stop_id = int(tokens[0])
            if dist(lat,lon,f_lat,f_lon) <= 300:
                my_list.append(stop_id)
    file.close()
    return my_list

def find_lat_long(stop_id):
    file = open('/content/drive/MyDrive/DPM/stops.csv','r')
    file.readline()
    while True:
        line = file.readline()
        if len(line) == 0:
            break;
        else:
            token = line.split(',')
            id = int(token[0])
            lat = float(token[-2])
            lon = float(token[-1])
            if id == int(stop_id):
                file.close()
                return (lat,lon)

def list_of_walks2(stop_id):
    lat, lon = find_lat_long(stop_id)
    file = open('/content/drive/MyDrive/DPM/stops.csv','r')
    my_list = []
    file.readline()
    while True:
        line = file.readline()
        if len(line)==0:
            break;
        else:
            tokens = line.split(',')
            f_lat = float(tokens[-2])
            f_lon = float(tokens[-1])
            stop_id = int(tokens[0])
            if dist(lat,lon,f_lat,f_lon) <= 300:
                my_list.append(stop_id)
    file.close()
    return my_list
            

            
def find_stop_id(stopname):
    file = open('/content/drive/MyDrive/DPM/stops.csv','r')
    file.readline()
    while True:
        line = file.readline()
        if len(line) == 0:
            break;
        else:
            tokens = line.split(',')
            id = int(tokens[0])
            name = tokens[2]
            if stopname == name:
                file.close()
                return id
    return -1 # If not found the stop_name!

In [4]:
# Convert a list of list to a single list
def convert(t):
    flat_list = [(x for x in item) if(type(item)==list) else item for item in t]
    flat_list = [x for x in flat_list if type(x)==int]
    flat_list = list(set(flat_list))
    return flat_list

def convert2(t):
    flat_list = [item for sublist in t for item in sublist]
    flat_list = list(set(flat_list))
    return flat_list

def remove_dup(t):
    return list(set(t))

import numpy as np
from geopy.distance import geodesic

def dist(x,y,a,b):
    return geodesic((x,y),(a,b)).m

In [5]:
# for a given trip_id, all the direct_routes in the trip
trip_id_dict = dict()
stop_times = open('/content/drive/MyDrive/DPM/stop_times.csv','r')
flag = False
while True:
    line = stop_times.readline()
    if len(line) == 0:
        break;
    tokens = line.split(',')
    if flag:
        if tokens[0] not in trip_id_dict.keys():
            trip_id_dict[tokens[0]] = [int(float((tokens[-2])))]
        else:
            trip_id_dict[tokens[0]].append(int(float((tokens[-2]))))
    else:
        flag = True;
    
stop_times.close()

In [6]:
# direct_path,  this function stores all the stop_ids that can be reached directly from a stop_id
load = False

if load:
    direct_path = load_data()
else:
    direct_path = dict()
    for trip_id in trip_id_dict:
        for stop_ids in trip_id_dict[trip_id]:
            if stop_ids in direct_path.keys():
                direct_path[stop_ids] = direct_path[stop_ids] + trip_id_dict[trip_id]
                direct_path[stop_ids] = list(set(direct_path[stop_ids]))
            else:
                direct_path[stop_ids] = trip_id_dict[trip_id]
    direct_path
    store_data()

In [8]:
# Unique stop ids are this much in full dataset
len(direct_path.keys())

9591

In [ ]:
# Find for each stop id in direct_path 300m stations
walks_dict = dict()
count = 0
for ids in direct_path:
    walks_dict[ids] = convert(list_of_walks2(ids))
    count = count+1
    if(count%500 == 0):
      print(count)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


In [ ]:
# Example of 300m walkable distance stop_ids from stop_id = 7812.
convert(list_of_walks2(7812))

[7812, 7655, 6473, 1208, 8889, 1658]

In [ ]:
# Append all walkable stops for a given stop_id in a given route!
for stop_id in direct_path:
    new_list = [walks_dict[direct_stop] for direct_stop in direct_path[stop_id]]
    new_list = convert2(new_list)
    direct_path[stop_id] = direct_path[stop_id]+new_list
    direct_path[stop_id] = set(direct_path[stop_id])
    #direct_path[stop_id] = convert(direct_path[stop_id])

In [ ]:
store_data()

In [ ]:
def exist(src_id,dst_id):
    for id in direct_path[src_id]:
        if id == dst_id:
            return True;
        else:
            continue;
    return False;

In [ ]:
source = input('Enter Source:-')
dest   = input('Enter Destination:-')
source_id = find_stop_id(source)
dest_id = find_stop_id(dest)
print(exist(source_id,dest_id))